<a href="https://colab.research.google.com/github/sisomimoctrl/study-pytorch/blob/main/dataset_dataloader_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

torch.manual_seed(777)

"""
    0: TensorDataset 사용, PyTorch 제공
    1: CumstomDataset 사용, base는 Dataset
"""
method = 1
print(f'method: {method}\n')

# 데이터
if method == 0:
    x_train = torch.FloatTensor([[73, 80, 75],
                                 [93, 88, 93],
                                 [89, 91, 90],
                                 [96, 98, 100],
                                 [73, 66, 70]])
    y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
    dataset = TensorDataset(x_train, y_train)
    
elif method == 1:
    class CustomDataset(Dataset):
        def __init__(self):
            self.x_data = [[73, 80, 75],
                           [93, 88, 93],
                           [89, 91, 90],
                           [96, 98, 100],
                           [73, 66, 70]]
            self.y_data = [[152], [185], [180], [196], [142]]
            
        # 총 데이터의 개수를 리턴
        def __len__(self): 
            return len(self.x_data)
        
        # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
        def __getitem__(self, idx): 
            x = torch.FloatTensor(self.x_data[idx])
            y = torch.FloatTensor(self.y_data[idx])
            return x, y

    dataset = CustomDataset()

    
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

class LinearRegressionModel(nn.Module):
    def __init__(self): #
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)
    
model = LinearRegressionModel()

# optimizer 설정. 경사 하강법 SGD를 사용하고 learning rate를 의미하는 lr은 1e-5
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

# 전체 훈련 데이터에 대해 경사 하강법을 3,000회 반복
nb_epochs = 3000
for epoch in range(nb_epochs+1):
    
    for batch_idx, samples in enumerate(dataloader):
        
        x_train, y_train = samples
        
        # H(x) 계산
        prediction = model(x_train)
        
        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 개선하는 부분
        # gradient를 0으로 초기화
        optimizer.zero_grad()
        # 비용 함수를 미분하여 gradient 계산
        cost.backward() # backward 연산
        # W와 b를 업데이트
        optimizer.step()
    
    if epoch % 300 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

        
# 검증과정

# 임의의 입력 [73, 80, 75]를 선언
new_var =  torch.FloatTensor([[73, 80, 75]]) 

# 파라미터 출력
pred_y = model(new_var)
print(f'\n{list(model.parameters())}\n')

print(f'훈련 후 입력이 {new_var}일 때의 예측값 : {pred_y.item():.6f}')

method: 1

Epoch    0/3000 Cost: 7772.315430
Epoch  300/3000 Cost: 1.412876
Epoch  600/3000 Cost: 1.917449
Epoch  900/3000 Cost: 1.200064
Epoch 1200/3000 Cost: 1.093712
Epoch 1500/3000 Cost: 2.243459
Epoch 1800/3000 Cost: 1.736664
Epoch 2100/3000 Cost: 0.356883
Epoch 2400/3000 Cost: 0.305195
Epoch 2700/3000 Cost: 0.751829
Epoch 3000/3000 Cost: 0.607684

[Parameter containing:
tensor([[0.8026, 0.4455, 0.7651]], requires_grad=True), Parameter containing:
tensor([-0.1051], requires_grad=True)]

훈련 후 입력이 tensor([[73., 80., 75.]])일 때의 예측값 : 151.510345
